Когда это вообще может быть нужно?

In [1]:
import requests
import json
from tqdm import tqdm
#from urllib import urlopen
from multiprocessing.dummy import Pool as ThreadPool
from IPython.display import display, HTML
from lxml import etree
#import grab
from bs4 import BeautifulSoup

In [2]:
BASH_URL = 'http://bash.org.ru'
resp = requests.get(BASH_URL)

In [3]:
resp.status_code

200

In [4]:
resp.ok

True

<a href="https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP">Список кодов состояния HTTP</a>

**TLDR**: 
- 2** - всё хорошо
- 3** - редирект
- 4** - проблема клиента (404 - не найдено, 403 - нет доступа)
- 5** - проблема сервера (пятисотки)

In [5]:
resp.text

'<!doctype html>\n<html id="godtier">\n<head>\n\t<title>Цитатник Рунета</title>\n\n\t\n\t<meta charset="windows-1251" />\n\n\t<link href="http://bash.im/reset.css" type="text/css" rel="stylesheet" />\n\t<meta name="format-detection" content="telephone=no">\n\t\t\t\t<meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=yes" />\n\t<link href="http://bash.im/bor25.css?20140303" type="text/css" rel="stylesheet" />\n\t\t\n\t\t<link rel="alternate" title="Цитатник Рунета - bash.im" href="http://bash.im/rss/" type="application/rss+xml">\n\n\t\t\t\t<link rel="next" href="/index/1350" />\t\t\n\t\n\t\n\t<script src="https://code.jquery.com/jquery-1.8.3.min.js" integrity="sha256-YcbK69I5IXQftf/mYD8WY0/KmEDCv1asggHpJk1trM8=" crossorigin="anonymous"></script>\n\t<script type="text/javascript" src="http://bash.im/jquery-ui-1.9.2.dp.min.js"></script>\n\t<link href="http://bash.im/jquery-ui-1.9.2.dp.min.css" rel="stylesheet" />\n\t<script type="text/javascript" src="http://

In [6]:
# HTML(resp.text)

## User-Agents

https://www.dafont.com/

<img src="https://www.ntu.edu.sg/home/ehchua/programming/webprogramming/images/HTTP_RequestMessageExample.png">

<img src="https://docs.microsoft.com/ru-ru/aspnet/web-api/overview/advanced/http-cookies/_static/image1.png">

In [7]:
resp = requests.get('https://www.dafont.com/')

In [8]:
resp.status_code

200

In [9]:
HTML(resp.text)

NameError: name 'HTML' is not defined

In [10]:
resp.text

''

In [11]:
from fake_useragent import UserAgent
ua = UserAgent()

headers = {
    'User-Agent': ua.chrome
}

ModuleNotFoundError: No module named 'fake_useragent'

In [12]:
resp = requests.get('https://www.dafont.com/', headers=headers)

NameError: name 'headers' is not defined

In [ ]:
resp.text

## API

In [14]:
requests.get('https://httpbin.org/get?my_arg=test').json()

{'args': {'my_arg': 'test'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Connection': 'close',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.18.4'},
 'origin': '188.44.42.220',
 'url': 'https://httpbin.org/get?my_arg=test'}

Хорошо бы из данных теперь что-то достать.

Регулярные выражения?..

<img src="https://i.pinimg.com/originals/f4/e8/35/f4e835a17ffc770a69f632d257b77473.png">

1) CSS-selectors

2) <a href="http://msiter.ru/tutorials/xpath/syntax">XPath</a>

3) Semantic (class_name)

In [18]:
resp = requests.get(BASH_URL)
bs = BeautifulSoup(resp.text, 'lxml')

Можем вытащить ссылки

In [20]:
for x in bs.find_all('a')[:10]:
    print (x.get('href'))

/
http://ithappens.me/
http://zadolba.li/
/
/random
/best
/byrating
/abyss
/abysstop
/abyssbest


Ссылки относительные. Не всегда это удобно. И не нужно их склеивать руками (вспоминаем про `os.path.join`)

In [24]:
from urlparse import urljoin, unquote, parse_qs

ModuleNotFoundError: No module named 'urlparse'

In [25]:
for x in bs.find_all('a')[:10]:
    print (urljoin(BASH_URL, x.get('href')))

NameError: name 'urljoin' is not defined

Можем сразу вытащить текст. Не идеально

In [26]:
print bs.text.replace('\n\n', '\n')

SyntaxError: invalid syntax (<ipython-input-26-e6adfdc83134>, line 1)

Простое решение, которое покроет 90% случаев:

In [ ]:
for x in bs('script'):
    x.extract()

In [ ]:
print bs.text.replace('\n\n', '\n')

Скорее всего, нам нужен не весь текст (Регистрация, Меню, etc)

In [ ]:
bs.find_all('div', 'text')[:3]

In [ ]:
for x in bs.find_all('div', 'text')[:3]:
    print x.text
    print '-' * 80

Похоже, мы потеряли переносы строк

In [ ]:
for x in bs.find_all('div', 'text')[:3]:
    print x
    print '-' * 80

In [ ]:
for br in bs.find_all("br"):
    br.replace_with("\n")

In [ ]:
for x in bs.find_all('div', 'text')[:3]:
    print x.text
    print '-' * 80

# unquote, parse_qs

In [ ]:
WIKI_URL = 'http://ru.wikipedia.org'
resp = requests.get(WIKI_URL)
bs = BeautifulSoup(resp.content, 'lxml')

In [ ]:
for x in bs.find_all('a')[:5]:
    if x.get('href'):
        print urljoin(WIKI_URL, x['href']), x.text
        print 

In [ ]:
for x in bs.find_all('a')[:5]:
    if x.get('href'):
        print unquote(urljoin(WIKI_URL, x['href'])), x.text
        print 

In [ ]:
for x in bs.find_all('a'):
    if x.get('href') and '?' in x.get('href'):
        print urljoin(WIKI_URL, x['href'])
        print unquote(urljoin(WIKI_URL, x['href']))
        print json.dumps(parse_qs(urljoin(WIKI_URL, x['href'])), indent=2, ensure_ascii=False)
        break

## Redirects

In [ ]:
r = requests.get('http://github.com', allow_redirects=False)
r.status_code

In [ ]:
r = requests.head('http://github.com', allow_redirects=True)
r.url

In [ ]:
r.history

Редиректы через js - это <a href="https://stackoverflow.com/a/21396837">больная боль</a>

## Social

Вы можете себе облегчить жизнь, если посмотрите, как изменяется путь и GET-параметры в зависимости от ваших действий

https://www.kinopoisk.ru/top/lists/1/filtr/v/sort/order/

In [ ]:
resp = requests.get('https://www.kinopoisk.ru/top/lists/1/filtr/v/sort/order/')
bs = BeautifulSoup(resp.text)
len([float(x.get('value')) for x in bs.find_all('div', 'WidgetStars')])

In [ ]:
resp = requests.get('https://www.kinopoisk.ru/top/lists/1/filtr/v/sort/order/perpage/200/')
bs = BeautifulSoup(resp.text)
len([float(x.get('value')) for x in bs.find_all('div', 'WidgetStars')])

In [ ]:
print bs.findAll('tr')[4].prettify()

In [ ]:
# HTML(resp.text)

In [ ]:
resp = requests.get('https://brandshop.ru/new/')
bs = BeautifulSoup(resp.text)
len(bs.find_all('div', 'product'))

In [ ]:
resp = requests.get('https://brandshop.ru/new/?limit=1000')
bs = BeautifulSoup(resp.text)
len(bs.find_all('div', 'product'))

Скачивание данных - i/o-операция. Большую часть времени ваш процессор не выполняет никаких задач, потому что ждёт, пока данные придут по сети

<img src="https://qph.ec.quoracdn.net/main-qimg-123fabd6813bdc59da5fe392d37b3314.webp">

In [27]:
start_post = 349000
end_post = 349200
for x in tqdm(range(start_post, end_post, 2)):
    resp = requests.get('https://habrahabr.ru/post/%d/' % x)

 17%|█▋        | 17/100 [00:11<00:57,  1.44it/s]

KeyboardInterrupt: 

In [28]:
%%time
t = ThreadPool(16)
r = t.map(lambda x: requests.get('https://habrahabr.ru/post/%d/' % x), 
          range(start_post, end_post, 2))
t.close()
t.join()

CPU times: user 3.07 s, sys: 479 ms, total: 3.55 s
Wall time: 8.05 s


### Selenium

https://sites.google.com/a/chromium.org/chromedriver/downloads

In [ ]:
BEATLES = 'https://music.yandex.ru/artist/35004/tracks'
resp = requests.get(BEATLES, headers=headers)
bs = BeautifulSoup(resp.content, 'lxml')

In [ ]:
bs.find_all('div', 'd-track')

In [ ]:
import re

In [ ]:
context_len = 200
for x in re.finditer('Yesterday', resp.content):
    print resp.content[x.start() - context_len : x.end() + context_len]
    print 

In [ ]:
len(bs.find_all('a', 'track__title'))

In [ ]:
track_set = {x.get('href') for x in bs.find_all('a', 'track__title')}
list(track_set)[:5]

In [ ]:
len(track_set)

In [ ]:
from selenium import webdriver

In [ ]:
driver = webdriver.Chrome('/Users/v.fedorenko/chromedriver')
driver.get(BEATLES)

In [ ]:
track_ids = set()
driver.execute_script("window.scrollTo(0, 0)")
height = driver.find_element_by_class_name('lightlist_tracks').size.get('height')
for step in tqdm(range(0, height+200, 300)):
    driver.execute_script("window.scrollTo(0, %s)" % step)
    driver.refresh()
    for x in driver.find_elements_by_class_name('d-track__title'):
        track_ids.add(x.get_attribute('href'))

In [ ]:
len(track_ids)

## Прокси

In [30]:
resp = requests.get('https://www.expressvpn.com/ru/what-is-my-ip')
bs = BeautifulSoup(resp.content, 'lxml')
print (bs.find('div', 'detail-value').text)

188.44.42.220


In [32]:
# https://hidemy.name/ru/proxy-list/?type=hs#list
proxies = {
    'http': 'http://5.9.78.89:3128',
    'https': 'https://5.9.78.89:3128'
}
resp = requests.get('https://www.expressvpn.com/ru/what-is-my-ip', proxies=proxies)
bs = BeautifulSoup(resp.content, 'lxml')
print (bs.find('div', 'detail-value').text)

5.9.78.89


## Cookies

In [ ]:
resp = requests.get('http://habrahabr.ru')
bs = BeautifulSoup(resp.content, 'lxml')
print bs.find('button', 'btn_navbar_user-dropdown')

In [ ]:
COOKIES = ''
cookies = dict(map(lambda x: x.strip().split('='), COOKIES.split(';')))
cookies

In [ ]:
resp = requests.get('http://habrahabr.ru', cookies=cookies)
bs = BeautifulSoup(resp.content, 'lxml')
print bs.find('button', 'btn_navbar_user-dropdown').get('title')

Некоторые советы:

- Сохраняйте страницы в БД, чтобы не перекачивать их с самого начала, есть ваш процесс упадёт
- Доставайте из страниц всю полезную информаицю или сохраняте оригинал html
- Надеемся на лучшее (можем найти нужный div, у которого уникальный class), готовимся к худшему (копируем или гуглим XPath-запросы) 
- Перед тем как краулить - потратье пару минут на то, чтобы поработать с сайтом
- Не забывайте, что ваша библиотека делает запросы не совсем так, как ваш браузер: может потребоваться передача всех заголовков (юзерагент, ваши куки)
- Прокси
- Многопоточность. Количество потоков определяется опытным путём
- Если много интерактивных элементов: selenium или пытаться по инструментам разработчика отследить логику запросов за дополнительным контентом
- Помнить, что мы можем обкачивать или много страниц с одного сайта, или много страниц с разных. Если качать с одного - проще заметить проблемы:
- - код ответа 200, но сообщение о блокировке IP
- - сайт говорит, что данные в одной кодировке, а на самом деле они в другой
- - подумать о нормализации урлов (нужно ли сохранять схему (http, https), нужно ли www.). Например, это разные урлы?
- - - `"ya.ru", "ya.ru/", "ya.ru/?", "http://ya.ru", "https://wwww.ya.ru", "ya.ru/?utm_source=vk", "ya.ru/?query=parsing",`